In [1]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 2.2 # distance between each cluster
unit = 'b' # unit of length
na = 2 # size of a cluster (monomer)
nc = 4 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = '6-31g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

# atoms = '''
# O  -1.551007  -0.114520   0.000000
# H  -1.934259   0.762503   0.000000
# H  -0.599677   0.040712   0.000000
# O   1.350625   0.111469   0.000000
# H   1.680398  -0.373741  -0.758561
# H   1.680398  -0.373741   0.758561
# '''
# basis = 'ccpvdz'

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo, _ = mf.stability()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#7309-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sat Dec 20 14:33:37 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

In [2]:
nfrozen = 0
# mycc = cc.CCSD(mf,frozen=nfrozen)
# mycc.kernel()[0]

In [3]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob
print(frag_lolist)



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 3.7396489642363  delta_f= 3.73965  |g|= 0.000998899  1 KF 1 Hx
macro= 2  f(x)= 3.7396489642363  delta_f= -1.77636e-15  |g|= 6.94017e-06  1 KF 1 Hx
macro X = 2  f(x)= 3.7396489642363  |g|= 6.94017e-06  4 intor 2 KF 2 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_low

In [4]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd
from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-5
run_frg_list = [0]
chol_cut = 1e-5

mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
# frag_res = [None] * nfrag
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)

print(lno_frozen[0])
print(lno_frozen[1])

[0.0001, 1e-05]
LO occ proj: 1 active | 0 standby | 3 orthogonal
LO occ proj: 0 active | 0 standby | 4 orthogonal
[ 0  1  2  7  8  9 10 11 12 13 14 15]
[ 0  1  8  9 10 11 12 13 14 15]


In [ ]:
print(uocc_loc[0],uocc_loc[0].size)

print(uocc_loc[1],uocc_loc[1].size)

[[1.]] 1
[] 0


In [ ]:
trial = 'uccsd_pt2'
print(trial)
trial += '_beta'
print(trial)

uccsd_pt2
uccsd_pt2_beta


In [8]:
prja = uocc_loc[0] @ uocc_loc[0].T.conj()
prjb = uocc_loc[1] @ uocc_loc[1].T.conj()
print(prja)
print(prjb)
prjlo = [prja, prjb]

[[1.]]
[[0. 0.]
 [0. 0.]]


In [8]:
(emp2,eccsd,ept), frag_msg = mlno.impurity_solve(mf, lno_coeff, uocc_loc, eris, frozen=lno_frozen)
print(emp2,eccsd,ept)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

-0.00047041037447406904 -0.0021417448196257806 -1.877488850268379e-06


In [9]:
mol = mf.mol
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in lno_frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in lno_frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in lno_frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in lno_frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)

ncas = (nactorb_a, nactorb_b)
ncore = (nfrzocc_a, nfrzocc_b)
nelec = (nactocc_a, nactocc_b)
print(ncas)
print(ncore)
print(nelec)

(4, 6)
(3, 2)
(1, 2)


In [10]:
mo_occ = mlno.mo_occ
# mlno.verbose_imp = 4
lno_frozen, maskact = ulnoccsd.get_maskact(lno_frozen, [mo_occ[0].size, mo_occ[1].size])
mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

(emp2,eccsd,ept,ecct1_0,ecct2_0), t1, t2 =\
    ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
# prjlo = [uocc_loc[0].T.conj(), uocc_loc[1].T.conj()]

prja = uocc_loc[0] @ uocc_loc[0].T.conj()
prjb = uocc_loc[1] @ uocc_loc[1].T.conj()
prjlo = [prja, prjb]

print(emp2,eccsd,ept,ecct1_0,ecct2_0)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

Init t2, MP2 energy = -0.000942899565727326

******** <class 'pyscf.lno.ulnoccsd.MODIFIED_UCCSD'> ********
CC2 = 0
CCSD nocc = (np.int64(1), np.int64(2)), nmo = (4, 6)
frozen orbitals [array([ 0,  1,  2,  7,  8,  9, 10, 11, 12, 13, 14, 15]), array([ 0,  1,  8,  9, 10, 11, 12, 13, 14, 15])]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-06
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 16000 MB (current use 977 MB)
Init E_corr(MODIFIED_UCCSD) = -0.000942899565727326
cycle = 1  E_corr(MODIFIED_UCCSD) = -0.00134883245029072  dE = -0.000405932885  norm(t1,t2) = 0.0144078
cycle = 2  E_corr(MODIFIED_UCCSD) = -0.00161849336533234  dE = -0.000269660915  norm(t1,t2) = 0.0120852
cycle = 3  E_corr(MODIFIED_UCCSD) = -0.00248876606611534  dE = -0.000870272701  norm(t1,t2) = 0.0

In [8]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt2_ad',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }

from mpi4py import MPI
if not MPI.Is_finalized():
    MPI.Finalize()

nelec, ncas = ulno_afqmc.prep_afqmc(
    mf,lno_coeff,t1,t2,lno_frozen,prjlo,
    options,chol_cut=chol_cut)

print(nelec, ncas)

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# Composing AO ERIs from DF basis
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (1, 2)
# Number of basis functions: (4, 6)
# Alpha Basis Cholesky shape: (184, 4, 4)
#  Beta Basis Cholesky shape: (184, 6, 6)
(1, 2) (4, 6)


In [3]:
from jax import numpy as jnp
import numpy as np

amp_file="amplitudes.npz"
mo_file="mo_coeff.npz"

amplitudes = np.load(amp_file)
t1a = jnp.array(amplitudes["t1a"])
t1b = jnp.array(amplitudes["t1b"])
t2aa = jnp.array(amplitudes["t2aa"])
t2ab = jnp.array(amplitudes["t2ab"])
t2bb = jnp.array(amplitudes["t2bb"])
prja = jnp.array(np.load(mo_file)["prja"])
prjb = jnp.array(np.load(mo_file)["prjb"])

In [6]:
print(t2aa.shape)
print(t2ab.shape)
print(t2bb.shape)
print(prja.shape)
print(prjb.shape)

(1, 3, 1, 3)
(1, 3, 2, 4)
(2, 4, 2, 4)
(1, 1)
(0, 2)


In [1]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
from ad_afqmc.lno_afqmc import ulno_afqmc
ham_data, prop, trial, wave_data, sampler, options, _ = (
    ulno_afqmc._prep_afqmc())

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (4, 6)
# nelec: (1, 2)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 98
# walker_type: uhf
# trial: uccsd_pt2_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# max_error: 0.0001
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#


In [ ]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])
# converged SCF energy = -4.01046204596874 

-4.010462045968735


In [15]:
import opt_einsum as oe
from jax import vmap
def calc_energy(trial,walker_up,walker_dn,h1,chol,wave_data):
    ene0 = ham_data["h0"]
    nocca, noccb = trial.nelec
    norba, norbb = trial.norb
    rot_h1a = h1[0][:nocca,:]
    rot_h1b = h1[1][:noccb,:]
    rot_chola = chol[0].reshape(-1,norba,norba)[:,:nocca,:]
    rot_cholb = chol[1].reshape(-1,norbb,norbb)[:,:noccb,:]
    greena, greenb = trial._calc_green(walker_up, walker_dn, wave_data)
    ene1 = jnp.sum(greena * rot_h1a) + jnp.sum(greenb * rot_h1b)
    f_up = oe.contract("gij,jk->gik", rot_chola, greena.T, backend="jax")
    f_dn = oe.contract("gij,jk->gik", rot_cholb, greenb.T, backend="jax")
    c_up = vmap(jnp.trace)(f_up)
    c_dn = vmap(jnp.trace)(f_dn)
    exc_up = jnp.sum(vmap(lambda x: x * x.T)(f_up))
    exc_dn = jnp.sum(vmap(lambda x: x * x.T)(f_dn))
    ene2 = (jnp.sum(c_up * c_up)
            + jnp.sum(c_dn * c_dn)
            + 2.0 * jnp.sum(c_up * c_dn)
            - exc_up - exc_dn) / 2.0

    return ene0 + ene1 + ene2

In [67]:
def prjmo(prj,s1e,mo):
    # prj and reconstruct mo
    # e.g. |B_p> = |A_q><A_q|B_p>
    #            = C^A_mq C^A(T)_qn|m><n|s> C^B_sp
    mo_rec = prj @ prj.T @ s1e @ mo
    return mo_rec

def common_las(mf, lno_coeff, ncas, ncore, torr=1e-5):
    print("# Constracting common LAS that span both Alpha and Beta active space")
    s1e = mf.get_ovlp()
    lno_acta = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
    lno_actb = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
    lno_actaa = lno_coeff[0].T @ s1e @ lno_acta
    lno_actba = lno_coeff[0].T @ s1e @ lno_actb
    m = np.hstack([lno_actaa,lno_actba])
    u,s,_ = np.linalg.svd(m)
    print(f'# Common Active Space Singular values: {s}')
    for idx in range(1,m.shape[1]+1):
        prj = lno_coeff[0] @ u[:,:idx]
        prj_acta = prjmo(prj,s1e,lno_actb)
        prj_actb = prjmo(prj,s1e,lno_acta)
        losa = abs(prj_acta-lno_actb).max()
        losb = abs(prj_actb-lno_acta).max()
        print(f"# Active space loss: ({losa:.2e}, {losb:.2e})")
        if losa < torr and losb < torr:
            break
    print(f"# Minimum number of SVD orbitals to span both Alpha and Beta active space: {idx}")
    # print(f"# Active space loss: ({losa:.2e}, {losb:.2e})")
    clas_coeff = lno_coeff[0] @ u[:,:idx]
    a2c = clas_coeff.T @ s1e @ lno_acta
    b2c = clas_coeff.T @ s1e @ lno_actb
    return clas_coeff, a2c, b2c

In [17]:
from pyscf import df, ao2mo, lib
nao = mol.nao
if getattr(mf, "with_df", None) is not None:
    # decompose eri in MO to achieve linear scale
    print("# Composing AO ERIs from DF basis")
    chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
    chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
    chol_df = chol_df.reshape((-1, nao, nao))
    # eri_ao = lib.einsum('lpr,lqs->prqs', chol_df, chol_df, optimize='optimal')
    # print("# Composing local active space ERIs from AO ERIs")
    # eri_lnoa = ao2mo.kernel(eri_ao,lno_acta,compact=False)
    # eri_lnoa = eri_lnoa.reshape(ncas[0]**2,ncas[0]**2)
    # eri_lnob = ao2mo.kernel(eri_ao,lno_actb,compact=False)
    # eri_lnob = eri_lnob.reshape(ncas[1]**2,ncas[1]**2)
    # print("# Decomposeing LNO-ERIs to Cholesky vectors")
    # chola = pyscf_interface.modified_cholesky(eri_lnoa,chol_cut)
    # cholb = pyscf_interface.modified_cholesky(eri_lnob,chol_cut)
    # chola = chola.reshape(-1,ncas[0],ncas[0])
    # cholb = cholb.reshape(-1,ncas[1],ncas[1])
    # print(f'# Alpha chol vector shape: {chola.shape}')
    # print(f'#  Beta chol vector shape: {cholb.shape}')
    # if chola.shape[0] != cholb.shape[0]:
    #     print("# Different number of chol vectors detected!")
    #     print("# Equalizing the number of chol vectors by adding zeros to the smaller one")
    #     chola, cholb = uchol_eql(chola, cholb)
    #     print(f'# New Alpha chol vector shape: {chola.shape}')
    #     print(f'# New  Beta chol vector shape: {cholb.shape}')
    # # FIX: find the space that just span active a and b
    # eri_clas = ao2mo.kernel(eri_ao,clas_coeff,compact=False)
    # eri_clas = eri_clas.reshape(nclas**2,nclas**2)
    # print("# Decomposing MO ERIs to Cholesky vectors")
    # print(f"# Cholesky cutoff is: {chol_cut}")
    # chol_clas = pyscf_interface.modified_cholesky(eri_clas,max_error=chol_cut)
    # chol_clas = chol_clas.reshape((-1, nclas, nclas))
    # chol_a = lib.einsum('pr,grs,sq->gpq',a2c.T,chol_clas,a2c)
    # chol_b = lib.einsum('pr,grs,sq->gpq',b2c.T,chol_clas,b2c)
    # # # a2b = <B|A>
    # # # <B|L|B> = <B|A><A|L|A><A|B>
    # # # transform Lb from La so they have the same number of vectors
    # # chol_b = jnp.einsum('pr,grs,sq->gpq',a2b,chol_a,a2b.T)
    # # chol_b = chol_b.reshape((-1, nao, nao))
    # # <Ap|L|Aq> = <Ap|mu><mu|L|nu><nu|Aq>
    chol_dfa = lib.einsum('pr,grs,sq->gpq',lno_coeff[0].T,chol_df,lno_coeff[0])
    chol_dfb = lib.einsum('pr,grs,sq->gpq',lno_coeff[1].T,chol_df,lno_coeff[1])
    chol_dfa = chol_dfa[:,ncore[0]:ncore[0]+ncas[0],ncore[0]:ncore[0]+ncas[0]]
    chol_dfb = chol_dfb[:,ncore[1]:ncore[1]+ncas[1],ncore[1]:ncore[1]+ncas[1]]

# Composing AO ERIs from DF basis


In [ ]:
from ad_afqmc import pyscf_interface

lno_acta = lno_coeff[0][:,ncore[0]:ncore[0]+ncas[0]]
lno_actb = lno_coeff[1][:,ncore[1]:ncore[1]+ncas[1]]
eri_ao = lib.einsum('lux,lvw->uxvw', chol_df, chol_df, optimize='optimal')
print("# Composing local active space ERIs from AO ERIs")
# eri_lnoa = ao2mo.kernel(eri_ao,lno_acta,compact=False)
eri_lnoa = lib.einsum("pu,xr,uxvw,qv,ws->prqs",lno_acta.T,lno_acta,eri_ao,lno_acta.T,lno_acta)
eri_lnoa = eri_lnoa.reshape(ncas[0]**2,ncas[0]**2)
# eri_lnob = ao2mo.kernel(eri_ao,lno_actb,compact=False)
eri_lnob = lib.einsum("pu,xr,uxvw,qv,ws->prqs",lno_actb.T,lno_actb,eri_ao,lno_actb.T,lno_actb)
eri_lnob = eri_lnob.reshape(ncas[1]**2,ncas[1]**2)
print("# Decomposeing LNO-ERIs to Cholesky vectors")
chol_lnoa = pyscf_interface.modified_cholesky(eri_lnoa,chol_cut)
chol_lnob = pyscf_interface.modified_cholesky(eri_lnob,chol_cut)
chol_lnoa = chol_lnoa.reshape(-1,ncas[0],ncas[0])
chol_lnob = chol_lnob.reshape(-1,ncas[1],ncas[1])
print(f'# Alpha chol vector shape: {chol_lnoa.shape}')
print(f'#  Beta chol vector shape: {chol_lnob.shape}')

# if chol_lnoa.shape[0] != chol_lnob.shape[0]:
#     print("# Different number of chol vectors detected!")
#     print("# Equalizing the number of chol vectors by adding zeros to the smaller one")
#     chol_lnoa, chol_lnob = uchol_eql(chol_lnoa, chol_lnob)
#     print(f'# New Alpha chol vector shape: {chol_lnoa.shape}')
#     print(f'# New  Beta chol vector shape: {chol_lnob.shape}')

# Composing local active space ERIs from AO ERIs
# Decomposeing LNO-ERIs to Cholesky vectors
# Alpha chol vector shape: (10, 4, 4)
#  Beta chol vector shape: (18, 6, 6)
# Different number of chol vectors detected!
# Equalizing the number of chol vectors by adding zeros to the smaller one
# New Alpha chol vector shape: (18, 4, 4)
# New  Beta chol vector shape: (18, 6, 6)


In [68]:
clas_coeff, a2c, b2c = common_las(mf, lno_coeff, ncas, ncore)

# Constracting common LAS that span both Alpha and Beta active space
# Common Active Space Singular values: [1.41420578 1.41399369 1.41310826 1.37143617 1.         1.
 0.34519969 0.05590221 0.02493692 0.00469082]
# Active space loss: (1.03e+00, 1.10e+00)
# Active space loss: (1.03e+00, 1.07e+00)
# Active space loss: (1.09e+00, 1.00e+00)
# Active space loss: (9.50e-01, 2.08e-01)
# Active space loss: (6.23e-01, 2.08e-01)
# Active space loss: (1.76e-01, 2.08e-01)
# Active space loss: (1.94e-02, 2.60e-02)
# Active space loss: (1.49e-02, 1.70e-02)
# Active space loss: (2.31e-03, 3.46e-03)
# Active space loss: (3.66e-15, 5.55e-15)
# Minimum number of SVD orbitals to span both Alpha and Beta active space: 10


In [73]:
chol_cut = 1e-8
nclas = clas_coeff.shape[1]
eri_clas = ao2mo.kernel(eri_ao,clas_coeff,compact=False)
eri_clas = eri_clas.reshape(nclas**2,nclas**2)
print("# Decomposing MO ERIs to Cholesky vectors")
print(f"# Cholesky cutoff is: {chol_cut}")
chol_clas = pyscf_interface.modified_cholesky(eri_clas,max_error=chol_cut)
chol_clas = chol_clas.reshape((-1, nclas, nclas))
chol_clasa = lib.einsum('pr,grs,sq->gpq',a2c.T,chol_clas,a2c)
chol_clasb = lib.einsum('pr,grs,sq->gpq',b2c.T,chol_clas,b2c)
print(chol_clasa.shape)
print(chol_clasb.shape)

# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-08
(37, 4, 4)
(37, 6, 6)


In [30]:
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
h1 = ham_data['h1']
chol = [chol_dfa,chol_dfb]
e_df = jnp.real(calc_energy(trial,walker_up,walker_dn,h1,chol,wave_data))
print(e_df)

-4.010462045968728


In [74]:
print(chol_clasa.shape)
print(chol_clasb.shape)
chol = [chol_clasa,chol_clasb]
e_lno = jnp.real(calc_energy(trial,walker_up,walker_dn,h1,chol,wave_data))
print(e_lno)

(37, 4, 4)
(37, 6, 6)
-4.0104620458204865


In [57]:
chol = [chol_clasa,chol_clasb]
e_lno = jnp.real(calc_energy(trial,walker_up,walker_dn,h1,chol,wave_data))
print(e_lno)

-4.010487339196528


In [65]:
import numpy as np
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)
# walker_up_r = wave_data['exp_t1a'] @ walker_up_r
# walker_dn_r = wave_data['exp_t1b'] @ walker_dn_r
te_ad, t_ad = _eorb_pt2_ad(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
te_tc, t_tc, _ = _eorb_pt2_tc(trial,walker_up_r,walker_dn_r,ham_data,wave_data)
print(te_ad, t_ad)
print(te_tc, t_tc)

-1.6636454781411971 0.20925761409113638
-1.6636454781411971 0.20925761409113633
